# Denoising Dirty Documents — Plan and Experiment Log

Objective
- Competition: denoising-dirty-documents
- Metric: RMSE on pixel intensities
- Output: submission.csv with id,value pairs (ids are image_index_pixel for test set).
- Non-negotiable rule: WIN A MEDAL.

Performance Targets
- Gold: RMSE ≤ 0.01794
- Silver: RMSE ≤ 0.02609
- Bronze: RMSE ≤ 0.04517

Data
- train/: noisy PNGs
- train_cleaned/: clean PNGs (same filenames) for supervised validation
- test/: noisy PNGs
- sampleSubmission.csv: id,value format

High-level Strategy
1) Establish fast, strong classical denoising baselines that historically perform well on this comp:
   - Median filtering (various kernel sizes)
   - Bilateral filter
   - Non-Local Means (fastNlMeans)
   - Morphological open/close and tophat/blackhat with adaptive thresholds
   - Simple background estimation via large-kernel median/gaussian and subtract/normalize
2) Validate on train vs train_cleaned using per-image RMSE.
3) Select per-image best method or learn a blending (linear regression) on the methods' outputs to minimize RMSE.
4) Optionally refine with tuned parameters per-image (grid over kernel sizes, sigma values) with early stopping.
5) Generate predictions for test and write submission.csv in correct order.

Why classical first?
- Fast, low-risk, strong on this dataset (text documents). Deep models add setup and time; we can add later if needed.

Milestones (request expert review at each):
A) Plan (this cell)
B) Data loading + EDA
C) Baseline methods implemented + CV on train
D) Model selection/blending results
E) Final test inference + submission

Experiment Log
- 00: Plan drafted. Next: implement loaders and quick RMSE evaluation for baseline filters.


In [ ]:
%pip -q install opencv-python-headless

In [ ]:
# Fix OpenCV libGL issue by forcing headless build and removing conflicting builds
%pip -q uninstall -y opencv-python opencv-contrib-python opencv-python-headless opencv-contrib-python-headless
%pip -q install --no-cache-dir opencv-python-headless==4.9.0.80
import cv2, sys
print('cv2 version:', cv2.__version__)
print('Python:', sys.version)

In [1]:
import os
import time
import math
import glob
import csv
from typing import Dict, Tuple, List

import numpy as np
from PIL import Image
from skimage import exposure, morphology, filters, restoration, util
from scipy.ndimage import median_filter

# Paths
TRAIN_DIR = 'train'
CLEAN_DIR = 'train_cleaned'
TEST_DIR = 'test'
SAMPLE_SUB = 'sampleSubmission.csv'
SUBMISSION_OUT = 'submission.csv'

# Utilities
def read_gray_float(path: str) -> np.ndarray:
    img = Image.open(path).convert('L')
    arr = np.asarray(img, dtype=np.float32) / 255.0
    return arr

def to_uint8(img: np.ndarray) -> np.ndarray:
    return np.clip(img * 255.0, 0, 255).astype(np.uint8)

def rmse(a: np.ndarray, b: np.ndarray) -> float:
    return float(np.sqrt(np.mean((a - b) ** 2)))

# Core processing blocks (cv2-free implementations)
def clahe_enhance(img01: np.ndarray, clip_limit: float = 0.01, tile_grid: Tuple[int, int] = (8, 8)) -> np.ndarray:
    # skimage's clip_limit is fraction of tiles; 0.01-0.03 is typical
    # For tile_grid we can approximate via kernel size; equalize_adapthist handles automatically
    res = exposure.equalize_adapthist(util.img_as_float(img01), clip_limit=clip_limit)
    return res.astype(np.float32)

def white_tophat(img01: np.ndarray, radius: int = 21) -> np.ndarray:
    selem = morphology.disk(radius)
    opened = morphology.opening(img01, selem)
    wt = img01 - opened
    return np.clip(wt, 0.0, 1.0).astype(np.float32)

def background_sub_gaussian(img01: np.ndarray, sigma: int = 41) -> np.ndarray:
    bg = filters.gaussian(img01, sigma=sigma, preserve_range=True)
    res = img01 - bg
    return np.clip(res, 0.0, 1.0).astype(np.float32)

def gentle_denoise(img01: np.ndarray, method: str = 'bilateral', **kwargs) -> np.ndarray:
    if method == 'bilateral':
        sigma_color = float(kwargs.get('sigmaColor', 0.10))
        sigma_spatial = float(kwargs.get('sigmaSpace', 5.0))
        den = restoration.denoise_bilateral(img01, sigma_color=sigma_color, sigma_spatial=sigma_spatial, channel_axis=None)
    elif method == 'tv':
        weight = float(kwargs.get('weight', 0.1))
        den = restoration.denoise_tv_chambolle(img01, weight=weight, channel_axis=None)
    else:
        ksize = int(kwargs.get('ksize', 3)) | 1
        den = median_filter(img01, size=ksize, mode='reflect')
    return np.clip(den, 0.0, 1.0).astype(np.float32)

def final_cleanup(img01: np.ndarray, median_ksize: int = 3) -> np.ndarray:
    med = median_filter(img01, size=(median_ksize | 1), mode='reflect')
    return np.clip(med, 0.0, 1.0).astype(np.float32)

# Robust Global Pipeline (per expert guidance)
def pipeline_robust(img01: np.ndarray,
                    bg_mode: str = 'tophat',
                    radius: int = 21,
                    gauss_sigma: int = 41,
                    clahe_clip: float = 0.01,
                    clahe_tile: Tuple[int, int] = (8,8),
                    denoise_method: str = 'bilateral',
                    bilateral_d: int = 9,  # unused in skimage impl
                    bilateral_sigmaColor: float = 0.10,
                    bilateral_sigmaSpace: float = 5.0,
                    nlm_h: float = 10.0,  # unused
                    final_median: int = 3) -> np.ndarray:
    if bg_mode == 'tophat':
        base = white_tophat(img01, radius=radius)
    else:
        base = background_sub_gaussian(img01, sigma=gauss_sigma)
    enh = clahe_enhance(base, clip_limit=clahe_clip, tile_grid=clahe_tile)
    if denoise_method == 'bilateral':
        den = gentle_denoise(enh, method='bilateral', sigmaColor=bilateral_sigmaColor, sigmaSpace=bilateral_sigmaSpace)
    elif denoise_method == 'tv':
        den = gentle_denoise(enh, method='tv', weight=0.12)
    else:
        den = gentle_denoise(enh, method='median', ksize=3)
    out = final_cleanup(den, median_ksize=final_median)
    return np.clip(out, 0.0, 1.0).astype(np.float32)

# Simple alternative pipeline (Gaussian BG subtraction)
def pipeline_gaussbg(img01: np.ndarray) -> np.ndarray:
    base = background_sub_gaussian(img01, sigma=51)
    enh = clahe_enhance(base, clip_limit=0.015, tile_grid=(8,8))
    den = gentle_denoise(enh, method='bilateral', sigmaColor=0.12, sigmaSpace=6.0)
    out = final_cleanup(den, median_ksize=3)
    return np.clip(out, 0.0, 1.0).astype(np.float32)

# CV evaluation
def list_image_ids(dir_path: str) -> List[str]:
    files = glob.glob(os.path.join(dir_path, '*.png'))
    ids = [os.path.splitext(os.path.basename(p))[0] for p in files]
    return sorted(ids, key=lambda x: int(x))

def evaluate_pipeline_on_train(pipeline_name: str = 'robust', n_show: int = 3) -> None:
    train_ids = list_image_ids(TRAIN_DIR)
    clean_ids = set(list_image_ids(CLEAN_DIR))
    ids = [i for i in train_ids if i in clean_ids]
    print(f'Train images found: {len(ids)}')
    t0 = time.time()
    rmses = []
    for idx, img_id in enumerate(ids):
        if idx % 10 == 0:
            elapsed = time.time() - t0
            print(f'Processing {idx}/{len(ids)} (elapsed {elapsed:.1f}s) ...', flush=True)
        noisy = read_gray_float(os.path.join(TRAIN_DIR, f'{img_id}.png'))
        clean = read_gray_float(os.path.join(CLEAN_DIR, f'{img_id}.png'))
        if pipeline_name == 'robust':
            pred = pipeline_robust(noisy)
        else:
            pred = pipeline_gaussbg(noisy)
        score = rmse(pred, clean)
        rmses.append(score)
    avg = float(np.mean(rmses)) if rmses else math.inf
    print(f'Average RMSE ({pipeline_name}): {avg:.6f}')

# Test inference + submission writer
def predict_test_images(pipeline_name: str = 'robust') -> Dict[str, np.ndarray]:
    ids = list_image_ids(TEST_DIR)
    preds = {}
    t0 = time.time()
    for i, img_id in enumerate(ids):
        if i % 5 == 0:
            print(f'Test {i}/{len(ids)} ... elapsed {time.time() - t0:.1f}s', flush=True)
        noisy = read_gray_float(os.path.join(TEST_DIR, f'{img_id}.png'))
        if pipeline_name == 'robust':
            pred = pipeline_robust(noisy)
        else:
            pred = pipeline_gaussbg(noisy)
        preds[img_id] = pred
    return preds

def write_submission_streaming(preds: Dict[str, np.ndarray], sample_path: str = SAMPLE_SUB, out_path: str = SUBMISSION_OUT) -> None:
    # Stream over sampleSubmission order and write values directly
    with open(sample_path, 'r') as fin, open(out_path, 'w', newline='') as fout:
        reader = csv.reader(fin)
        writer = csv.writer(fout)
        header = next(reader)
        writer.writerow(header)
        line_ct = 0
        for row in reader:
            id_str = row[0]
            parts = id_str.split('_')
            if len(parts) != 3:
                continue
            img_id, r_str, c_str = parts
            # Sample file is 1-based indexed
            r = int(r_str) - 1
            c = int(c_str) - 1
            arr = preds.get(img_id)
            if arr is None:
                raise KeyError(f'Missing prediction for image id {img_id}')
            if r < 0 or c < 0 or r >= arr.shape[0] or c >= arr.shape[1]:
                raise IndexError(f'Index out of bounds for {img_id}: ({r},{c}) vs shape {arr.shape}')
            val = float(np.clip(arr[r, c], 0.0, 1.0))
            writer.writerow([id_str, f'{val:.6f}'])
            line_ct += 1
            if line_ct % 1000000 == 0:
                print(f'Wrote {line_ct} rows ...', flush=True)
    print(f'Submission written to {out_path}')

print('Setup complete (cv2-free). Next steps:')
print('- Run evaluate_pipeline_on_train("robust") to gauge baseline RMSE')
print('- If acceptable, run preds = predict_test_images("robust"); then write_submission_streaming(preds)')

Setup complete (cv2-free). Next steps:
- Run evaluate_pipeline_on_train("robust") to gauge baseline RMSE
- If acceptable, run preds = predict_test_images("robust"); then write_submission_streaming(preds)


In [ ]:
%pip -q install scikit-image imageio

In [ ]:
import time
t0 = time.time()
print('Evaluating pipeline: robust')
evaluate_pipeline_on_train('robust')
print(f'Elapsed: {time.time()-t0:.1f}s')

t1 = time.time()
print('\nEvaluating pipeline: gaussbg')
evaluate_pipeline_on_train('gaussbg')
print(f'Elapsed: {time.time()-t1:.1f}s')

In [ ]:
# Define Sauvola-based pipeline without extra imports (use skimage.filters from cell 3)
def pipeline_sauvola(img01: np.ndarray,
                      bg_mode: str = 'gauss',
                      radius: int = 25,
                      gauss_sigma: int = 45,
                      clahe_clip: float = 0.012,
                      sauvola_window: int = 31,
                      sauvola_k: float = 0.34,
                      unsharp_radius: float = 1.5,
                      unsharp_amount: float = 1.0) -> np.ndarray:
    # Background/illumination correction
    if bg_mode == 'tophat':
        base = white_tophat(img01, radius=radius)
    else:
        base = background_sub_gaussian(img01, sigma=gauss_sigma)
    # Contrast enhancement
    enh = clahe_enhance(base, clip_limit=clahe_clip)
    # Gentle sharpening to crispen text edges
    sharp = filters.unsharp_mask(enh, radius=unsharp_radius, amount=unsharp_amount, preserve_range=True)
    sharp = np.clip(sharp, 0.0, 1.0).astype(np.float32)
    # Sauvola adaptive threshold (text is dark -> background should be True/1)
    window = sauvola_window | 1
    thr = filters.threshold_sauvola(sharp, window_size=window, k=sauvola_k)
    binary_bg_white = (sharp > thr).astype(np.float32)  # background=1, text=0
    return binary_bg_white

def evaluate_custom_pipeline(func, max_images: int = 20) -> float:
    ids = list_image_ids(TRAIN_DIR)
    clean_ids = set(list_image_ids(CLEAN_DIR))
    ids = [i for i in ids if i in clean_ids]
    if max_images is not None:
        ids = ids[:max_images]
    print(f'Evaluating custom pipeline on {len(ids)} images ...')
    t0 = time.time()
    scores = []
    for idx, img_id in enumerate(ids):
        if idx % 10 == 0:
            print(f'  {idx}/{len(ids)} elapsed {time.time()-t0:.1f}s', flush=True)
        noisy = read_gray_float(os.path.join(TRAIN_DIR, f'{img_id}.png'))
        clean = read_gray_float(os.path.join(CLEAN_DIR, f'{img_id}.png'))
        pred = func(noisy)
        scores.append(rmse(pred, clean))
    avg = float(np.mean(scores)) if scores else math.inf
    print(f'Custom pipeline RMSE: {avg:.6f} (elapsed {time.time()-t0:.1f}s)')
    return avg

print('New pipeline added: pipeline_sauvola(). Use evaluate_custom_pipeline(lambda img: pipeline_sauvola(img)) to test.')

In [ ]:
print('Evaluating Sauvola pipeline on subset (15 images) ...')
subset_rmse = evaluate_custom_pipeline(lambda img: pipeline_sauvola(img), max_images=15)
print('Subset RMSE:', subset_rmse)

In [ ]:
from skimage.filters import threshold_otsu

def pipeline_div_otsu(img01: np.ndarray, sigma: int = 41) -> np.ndarray:
    # Divide normalization so background ~1.0, text darker
    bg = filters.gaussian(img01, sigma=sigma, preserve_range=True)
    corrected = img01 / (bg + 1e-3)
    # Robust rescale
    p1, p99 = np.percentile(corrected, 1), np.percentile(corrected, 99)
    corrected = exposure.rescale_intensity(corrected, in_range=(p1, p99), out_range=(0, 1)).astype(np.float32)
    thr = threshold_otsu(corrected)
    binary = (corrected > thr)
    # Cleanup
    binary = morphology.remove_small_holes(binary, area_threshold=64)
    binary = morphology.remove_small_objects(binary, min_size=16)
    return binary.astype(np.float32)

def pipeline_sub_inv_sauvola(img01: np.ndarray, sigma: int = 41, window: int = 31, k: float = 0.25) -> np.ndarray:
    # Subtract background, invert to get white background, then Sauvola
    bg = filters.gaussian(img01, sigma=sigma, preserve_range=True)
    corrected = img01 - bg  # residual; text negative
    inv = -corrected  # invert so background white
    p1, p99 = np.percentile(inv, 1), np.percentile(inv, 99)
    inv = exposure.rescale_intensity(inv, in_range=(p1, p99), out_range=(0, 1)).astype(np.float32)
    thr = filters.threshold_sauvola(inv, window_size=(window | 1), k=k)
    binary = (inv > thr)
    # Cleanup
    binary = morphology.remove_small_holes(binary, area_threshold=64)
    binary = morphology.remove_small_objects(binary, min_size=16)
    return binary.astype(np.float32)

# Quick subset eval for the two corrected pipelines
print('Evaluating divide+Otsu on 15 images ...')
rmse_div = evaluate_custom_pipeline(lambda img: pipeline_div_otsu(img), max_images=15)
print('Evaluating sub+invert+Sauvola on 15 images ...')
rmse_sau = evaluate_custom_pipeline(lambda img: pipeline_sub_inv_sauvola(img), max_images=15)
print('Subset RMSEs -> div+Otsu:', rmse_div, '| sub+inv+Sauvola:', rmse_sau)

In [ ]:
# Sanity checks + continuous pipelines (no hard threshold)
def baseline_sanity(max_images: int = 15):
    ids = list_image_ids(TRAIN_DIR)
    clean_ids = set(list_image_ids(CLEAN_DIR))
    ids = [i for i in ids if i in clean_ids][:max_images]
    r_noisy, r_white, r_black = [], [], []
    for img_id in ids:
        noisy = read_gray_float(os.path.join(TRAIN_DIR, f'{img_id}.png'))
        clean = read_gray_float(os.path.join(CLEAN_DIR, f'{img_id}.png'))
        r_noisy.append(rmse(noisy, clean))
        r_white.append(rmse(np.ones_like(clean, dtype=np.float32), clean))
        r_black.append(rmse(np.zeros_like(clean, dtype=np.float32), clean))
    print('Baseline RMSE (subset): noisy:', float(np.mean(r_noisy))), print('white:', float(np.mean(r_white))), print('black:', float(np.mean(r_black)))

def pipeline_div_continuous(img01: np.ndarray, sigma: int = 41, tv_weight: float = 0.08) -> np.ndarray:
    bg = filters.gaussian(img01, sigma=sigma, preserve_range=True)
    corrected = img01 / (bg + 1e-3)
    p1, p99 = np.percentile(corrected, 1), np.percentile(corrected, 99)
    corrected = exposure.rescale_intensity(corrected, in_range=(p1, p99), out_range=(0, 1)).astype(np.float32)
    den = restoration.denoise_tv_chambolle(corrected, weight=tv_weight, channel_axis=None)
    return np.clip(den.astype(np.float32), 0.0, 1.0)

def pipeline_sub_invert_continuous(img01: np.ndarray, sigma: int = 41, tv_weight: float = 0.08) -> np.ndarray:
    bg = filters.gaussian(img01, sigma=sigma, preserve_range=True)
    resid = img01 - bg
    inv = 1.0 - resid  # invert polarity so background tends to white
    p1, p99 = np.percentile(inv, 1), np.percentile(inv, 99)
    inv = exposure.rescale_intensity(inv, in_range=(p1, p99), out_range=(0, 1)).astype(np.float32)
    den = restoration.denoise_tv_chambolle(inv, weight=tv_weight, channel_axis=None)
    return np.clip(den.astype(np.float32), 0.0, 1.0)

print('Running sanity baselines on 15 images ...')
baseline_sanity(15)
print('\nEvaluating continuous divide pipeline on 15 images ...')
rmse_div_cont = evaluate_custom_pipeline(lambda img: pipeline_div_continuous(img), max_images=15)
print('Evaluating continuous sub-invert pipeline on 15 images ...')
rmse_sub_cont = evaluate_custom_pipeline(lambda img: pipeline_sub_invert_continuous(img), max_images=15)
print('Subset RMSEs -> div_cont:', rmse_div_cont, '| sub_invert_cont:', rmse_sub_cont)

In [ ]:
# Simple continuous denoising pipelines (no BG subtraction/binarization)
def pipeline_identity(img01: np.ndarray) -> np.ndarray:
    return img01.astype(np.float32)

def pipeline_median3(img01: np.ndarray) -> np.ndarray:
    return np.clip(median_filter(img01, size=3, mode='reflect').astype(np.float32), 0.0, 1.0)

def pipeline_median5(img01: np.ndarray) -> np.ndarray:
    return np.clip(median_filter(img01, size=5, mode='reflect').astype(np.float32), 0.0, 1.0)

def pipeline_bilateral_soft(img01: np.ndarray) -> np.ndarray:
    den = restoration.denoise_bilateral(img01, sigma_color=0.06, sigma_spatial=3.0, channel_axis=None)
    return np.clip(den.astype(np.float32), 0.0, 1.0)

def pipeline_tv_soft(img01: np.ndarray) -> np.ndarray:
    den = restoration.denoise_tv_chambolle(img01, weight=0.06, channel_axis=None)
    return np.clip(den.astype(np.float32), 0.0, 1.0)

def pipeline_clahe_median(img01: np.ndarray) -> np.ndarray:
    enh = exposure.equalize_adapthist(util.img_as_float(img01), clip_limit=0.01).astype(np.float32)
    den = median_filter(enh, size=3, mode='reflect')
    return np.clip(den.astype(np.float32), 0.0, 1.0)

def eval_pipelines_subset(max_images: int = 15):
    pipes = {
        'identity': pipeline_identity,
        'median3': pipeline_median3,
        'median5': pipeline_median5,
        'bilateral_soft': pipeline_bilateral_soft,
        'tv_soft': pipeline_tv_soft,
        'clahe_median': pipeline_clahe_median,
    }
    ids = list_image_ids(TRAIN_DIR)
    clean_ids = set(list_image_ids(CLEAN_DIR))
    ids = [i for i in ids if i in clean_ids][:max_images]
    results = {}
    for name, func in pipes.items():
        t0 = time.time()
        scores = []
        for idx, img_id in enumerate(ids):
            if idx % 10 == 0:
                print(f'{name}: {idx}/{len(ids)} elapsed {time.time()-t0:.1f}s', flush=True)
            noisy = read_gray_float(os.path.join(TRAIN_DIR, f'{img_id}.png'))
            clean = read_gray_float(os.path.join(CLEAN_DIR, f'{img_id}.png'))
            pred = func(noisy)
            scores.append(rmse(pred, clean))
        results[name] = float(np.mean(scores)) if scores else math.inf
        print(f'{name}: RMSE {results[name]:.6f} (elapsed {time.time()-t0:.1f}s)')
    print('Subset RMSE summary:', results)
    return results

print('Ready: call eval_pipelines_subset(15) to compare simple continuous denoisers.')

In [ ]:
print('Comparing simple continuous denoisers on 15 images ...')
results = eval_pipelines_subset(15)
best_name = min(results, key=results.get)
print('Best pipeline on subset:', best_name, 'RMSE:', results[best_name])

In [ ]:
# Generate test predictions with best quick pipeline (bilateral_soft) and write submission
import time
print('Predicting test images with bilateral_soft ...')
test_ids = list_image_ids(TEST_DIR)
preds = {}
t0 = time.time()
for i, img_id in enumerate(test_ids):
    if i % 5 == 0:
        print(f'  {i}/{len(test_ids)} elapsed {time.time()-t0:.1f}s', flush=True)
    noisy = read_gray_float(os.path.join(TEST_DIR, f'{img_id}.png'))
    pred = pipeline_bilateral_soft(noisy)
    preds[img_id] = np.clip(pred.astype(np.float32), 0.0, 1.0)
print('Writing submission.csv ...')
write_submission_streaming(preds, sample_path=SAMPLE_SUB, out_path=SUBMISSION_OUT)
print('Done.')

In [ ]:
# Linear blending of simple denoisers (Ridge) with leak-free image-wise CV and test inference
from sklearn.linear_model import Ridge
import random
import csv

# Expanded set of diverse pipelines (lighter set to keep runtime reasonable)
BLEND_PIPES = {
    'identity': pipeline_identity,
    'bilateral_soft': pipeline_bilateral_soft,
    'tv_soft': pipeline_tv_soft,
    'median3': pipeline_median3,
    'gaussian_07': pipeline_gaussian_07,
    'bilateral_soft2': pipeline_bilateral_soft2,
    'tv_w003': pipeline_tv_w003,
    'nlm_fast': pipeline_nlm_fast,
    'median5': pipeline_median5,
    'local_mean3': pipeline_local_mean3,
}

def compute_pipe_outputs(img: np.ndarray, pipes: dict) -> dict:
    out = {}
    for name, fn in pipes.items():
        out[name] = fn(img).astype(np.float32)
    return out

def sample_pixels(h: int, w: int, n_samples: int) -> np.ndarray:
    total = h * w
    n = min(n_samples, total)
    idx = np.random.choice(total, size=n, replace=False)
    return idx

def build_train_samples(pipes: dict, n_per_image: int = 4000, seed: int = 42):
    random.seed(seed); np.random.seed(seed)
    ids = list_image_ids(TRAIN_DIR)
    clean_ids = set(list_image_ids(CLEAN_DIR))
    ids = [i for i in ids if i in clean_ids]
    X_list, y_list, meta = [], [], []
    image_indices = []  # row -> image index mapping
    print(f'Building samples from {len(ids)} train images ...')
    t0 = time.time()
    for k, img_id in enumerate(ids):
        if k % 10 == 0: print(f'  {k}/{len(ids)} elapsed {time.time()-t0:.1f}s', flush=True)
        noisy = read_gray_float(os.path.join(TRAIN_DIR, f'{img_id}.png'))
        clean = read_gray_float(os.path.join(CLEAN_DIR, f'{img_id}.png'))
        outs = compute_pipe_outputs(noisy, pipes)
        h, w = clean.shape
        flat_idx = sample_pixels(h, w, n_per_image)
        feats = []
        for name in pipes.keys():
            feats.append(outs[name].reshape(-1)[flat_idx])
        Xi = np.stack(feats, axis=1)
        yi = clean.reshape(-1)[flat_idx]
        X_list.append(Xi.astype(np.float32))
        y_list.append(yi.astype(np.float32))
        image_indices.append(np.full(Xi.shape[0], k, dtype=np.int32))
        meta.append({'id': img_id, 'shape': (h, w)})
    X = np.concatenate(X_list, axis=0).astype(np.float32)
    y = np.concatenate(y_list, axis=0).astype(np.float32)
    image_indices = np.concatenate(image_indices, axis=0).astype(np.int32)
    return ids, meta, X, y, image_indices

def ridge_cv_and_fit(pipes: dict, k_folds: int = 5, n_per_image: int = 4000, alpha: float = 0.1):
    ids, meta, X, y, img_idx_map = build_train_samples(pipes, n_per_image=n_per_image, seed=42)
    # Prepare folds by image index (contiguous split for determinism)
    n_images = len(ids)
    idxs = np.arange(n_images)
    folds = []
    fold_sizes = [n_images // k_folds + (1 if i < (n_images % k_folds) else 0) for i in range(k_folds)]
    start = 0
    for fs in fold_sizes:
        folds.append(idxs[start:start+fs])
        start += fs
    cv_scores = []
    for f, val_img_idx in enumerate(folds):
        print(f'Fold {f+1}/{k_folds}: training Ridge(alpha={alpha}) ...', flush=True)
        train_img_idx = np.setdiff1d(idxs, val_img_idx, assume_unique=True)
        train_rows_mask = np.isin(img_idx_map, train_img_idx)
        tr_rows = np.where(train_rows_mask)[0]
        model = Ridge(alpha=alpha, fit_intercept=True)
        model.fit(X[tr_rows], y[tr_rows])
        # Validate by predicting full images of val set via weighted sum
        fold_rmses = []
        for vi in val_img_idx:
            img_id = ids[vi]
            noisy = read_gray_float(os.path.join(TRAIN_DIR, f'{img_id}.png'))
            clean = read_gray_float(os.path.join(CLEAN_DIR, f'{img_id}.png'))
            outs = compute_pipe_outputs(noisy, pipes)
            names = list(pipes.keys())
            coefs = model.coef_
            intercept = float(model.intercept_)
            pred = intercept
            for j, name in enumerate(names):
                pred = pred + coefs[j] * outs[name]
            pred = np.clip(pred, 0.0, 1.0).astype(np.float32)
            fold_rmses.append(rmse(pred, clean))
        cv_scores.append(float(np.mean(fold_rmses)))
        print(f'  Fold {f+1} RMSE: {cv_scores[-1]:.6f}', flush=True)
    print('CV RMSE scores:', cv_scores, 'Avg:', float(np.mean(cv_scores)))
    # Fit final model on all samples
    final_model = Ridge(alpha=alpha, fit_intercept=True)
    final_model.fit(X, y)
    print('Final model coef:', final_model.coef_, 'intercept:', float(final_model.intercept_))
    return final_model

def predict_test_with_model(model, pipes: dict) -> Dict[str, np.ndarray]:
    test_ids = list_image_ids(TEST_DIR)
    preds = {}
    names = list(pipes.keys())
    coefs = model.coef_
    intercept = float(model.intercept_)
    t0 = time.time()
    for i, img_id in enumerate(test_ids):
        if i % 5 == 0: print(f'Test blend {i}/{len(test_ids)} elapsed {time.time()-t0:.1f}s', flush=True)
        noisy = read_gray_float(os.path.join(TEST_DIR, f'{img_id}.png'))
        outs = compute_pipe_outputs(noisy, pipes)
        pred = intercept
        for j, name in enumerate(names):
            pred = pred + coefs[j] * outs[name]
        preds[img_id] = np.clip(pred, 0.0, 1.0).astype(np.float32)
    return preds

def write_submission_streaming_prec(preds: Dict[str, np.ndarray], sample_path: str = SAMPLE_SUB, out_path: str = 'submission_blend_leakfix.csv', precision: int = 4) -> None:
    fmt = '{:.' + str(precision) + 'f}'
    with open(sample_path, 'r') as fin, open(out_path, 'w', newline='') as fout:
        reader = csv.reader(fin)
        writer = csv.writer(fout)
        header = next(reader)
        writer.writerow(header)
        line_ct = 0
        for row in reader:
            id_str = row[0]
            parts = id_str.split('_')
            if len(parts) != 3:
                continue
            img_id, r_str, c_str = parts
            r = int(r_str) - 1
            c = int(c_str) - 1
            arr = preds.get(img_id)
            if arr is None:
                raise KeyError(f'Missing prediction for image id {img_id}')
            val = float(np.clip(arr[r, c], 0.0, 1.0))
            writer.writerow([id_str, fmt.format(val)])
            line_ct += 1
            if line_ct % 1000000 == 0:
                print(f'Wrote {line_ct} rows ...', flush=True)
    print(f'Submission written to {out_path}')

print('Training Ridge blend with leak-free image-wise CV ...')
# Reduce n_per_image a bit to accommodate more features while keeping runtime manageable
model = ridge_cv_and_fit(BLEND_PIPES, k_folds=5, n_per_image=2000, alpha=0.1)
print('Predicting test with blended model ...')
preds_blend = predict_test_with_model(model, BLEND_PIPES)
print('Writing blended submission (4-decimal precision) ...')
write_submission_streaming_prec(preds_blend, sample_path=SAMPLE_SUB, out_path='submission_blend_leakfix.csv', precision=4)
print('Blended submission written to submission_blend_leakfix.csv')

In [ ]:
# Prepare final submission.csv from leak-fixed blend
import shutil, os
src = 'submission_blend_leakfix.csv'
dst = 'submission.csv'
if not os.path.exists(src):
    raise FileNotFoundError(src)
shutil.copyfile(src, dst)
print('Copied', src, '->', dst, 'Size:', os.path.getsize(dst)/1e6, 'MB')

In [ ]:
# Additional denoising pipelines for richer blending features
from skimage import restoration as sk_restoration
from scipy.ndimage import gaussian_filter, uniform_filter

def pipeline_gaussian_07(img01: np.ndarray) -> np.ndarray:
    den = gaussian_filter(img01, sigma=0.7, mode='reflect')
    return np.clip(den.astype(np.float32), 0.0, 1.0)

def pipeline_bilateral_soft2(img01: np.ndarray) -> np.ndarray:
    den = sk_restoration.denoise_bilateral(img01, sigma_color=0.08, sigma_spatial=4.0, channel_axis=None)
    return np.clip(den.astype(np.float32), 0.0, 1.0)

def pipeline_bilateral_strong(img01: np.ndarray) -> np.ndarray:
    den = sk_restoration.denoise_bilateral(img01, sigma_color=0.12, sigma_spatial=6.0, channel_axis=None)
    return np.clip(den.astype(np.float32), 0.0, 1.0)

def pipeline_tv_w003(img01: np.ndarray) -> np.ndarray:
    den = sk_restoration.denoise_tv_chambolle(img01, weight=0.03, channel_axis=None)
    return np.clip(den.astype(np.float32), 0.0, 1.0)

def pipeline_tv_w010(img01: np.ndarray) -> np.ndarray:
    den = sk_restoration.denoise_tv_chambolle(img01, weight=0.10, channel_axis=None)
    return np.clip(den.astype(np.float32), 0.0, 1.0)

def pipeline_nlm_fast(img01: np.ndarray) -> np.ndarray:
    # h tuned to be gentle; adjust if needed
    den = sk_restoration.denoise_nl_means(img01, h=0.08, patch_size=3, patch_distance=5, fast_mode=True, preserve_range=True)
    return np.clip(den.astype(np.float32), 0.0, 1.0)

def pipeline_clahe_lowclip(img01: np.ndarray) -> np.ndarray:
    enh = exposure.equalize_adapthist(util.img_as_float(img01), clip_limit=0.015).astype(np.float32)
    return np.clip(enh, 0.0, 1.0)

def pipeline_local_mean3(img01: np.ndarray) -> np.ndarray:
    den = uniform_filter(img01, size=3, mode='reflect')
    return np.clip(den.astype(np.float32), 0.0, 1.0)

In [3]:
%pip -q install torch torchvision --index-url https://download.pytorch.org/whl/cu121
import torch, torchvision
print('Torch:', torch.__version__, 'CUDA available:', torch.cuda.is_available())

Torch: 2.5.1+cu121 CUDA available: True


In [4]:
# DnCNN denoiser: dataset, model, AMP training loop, validation, and tiled inference
import math, random, os, time
from typing import List, Tuple
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_train_arrays() -> Tuple[List[str], List[np.ndarray], List[np.ndarray]]:
    ids = list_image_ids(TRAIN_DIR)
    clean_ids = set(list_image_ids(CLEAN_DIR))
    ids = [i for i in ids if i in clean_ids]
    noisy_list, clean_list = [], []
    t0 = time.time()
    for k, img_id in enumerate(ids):
        if k % 20 == 0:
            print(f'  preload {k}/{len(ids)} elapsed {time.time()-t0:.1f}s', flush=True)
        noisy = read_gray_float(os.path.join(TRAIN_DIR, f'{img_id}.png'))
        clean = read_gray_float(os.path.join(CLEAN_DIR, f'{img_id}.png'))
        noisy_list.append(noisy.astype(np.float32))
        clean_list.append(clean.astype(np.float32))
    return ids, noisy_list, clean_list

def split_ids(ids: List[str], val_frac: float = 0.1, seed: int = 42) -> Tuple[List[int], List[int]]:
    rng = np.random.default_rng(seed)
    idx = np.arange(len(ids))
    rng.shuffle(idx)
    n_val = max(1, int(len(ids) * val_frac))
    val_idx = idx[:n_val].tolist()
    tr_idx = idx[n_val:].tolist()
    return tr_idx, val_idx

def random_crop_pair(noisy: np.ndarray, clean: np.ndarray, size: int) -> Tuple[np.ndarray, np.ndarray]:
    h, w = noisy.shape
    if h < size or w < size:
        pad_h = max(0, size - h)
        pad_w = max(0, size - w)
        noisy = np.pad(noisy, ((0, pad_h), (0, pad_w)), mode='reflect')
        clean = np.pad(clean, ((0, pad_h), (0, pad_w)), mode='reflect')
        h, w = noisy.shape
    y = random.randint(0, h - size)
    x = random.randint(0, w - size)
    return noisy[y:y+size, x:x+size], clean[y:y+size, x:x+size]

def augment_pair(a: np.ndarray, b: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    k = random.randint(0, 3)
    a = np.rot90(a, k).copy()
    b = np.rot90(b, k).copy()
    if random.random() < 0.5:
        a = np.fliplr(a).copy(); b = np.fliplr(b).copy()
    if random.random() < 0.5:
        a = np.flipud(a).copy(); b = np.flipud(b).copy()
    return a, b

class PatchDataset(Dataset):
    def __init__(self, noisy_list: List[np.ndarray], clean_list: List[np.ndarray], patch_size: int = 128, length: int = 20000):
        self.noisy = noisy_list
        self.clean = clean_list
        self.patch = patch_size
        self.length = length
    def __len__(self):
        return self.length
    def __getitem__(self, idx):
        i = random.randint(0, len(self.noisy) - 1)
        n, c = random_crop_pair(self.noisy[i], self.clean[i], self.patch)
        n, c = augment_pair(n, c)
        n = torch.from_numpy(n).unsqueeze(0)  # 1,H,W
        c = torch.from_numpy(c).unsqueeze(0)
        return n, c

class DnCNN(nn.Module):
    def __init__(self, depth: int = 15, n_feats: int = 64, in_ch: int = 1, out_ch: int = 1):
        super().__init__()
        layers = []
        layers += [nn.Conv2d(in_ch, n_feats, 3, 1, 1), nn.ReLU(inplace=True)]
        for _ in range(depth - 2):
            layers += [nn.Conv2d(n_feats, n_feats, 3, 1, 1), nn.BatchNorm2d(n_feats), nn.ReLU(inplace=True)]
        layers += [nn.Conv2d(n_feats, out_ch, 3, 1, 1)]
        self.net = nn.Sequential(*layers)
    def forward(self, x):
        noise = self.net(x)
        return x - noise  # residual learning

class CharbonnierLoss(nn.Module):
    def __init__(self, eps: float = 1e-3):
        super().__init__()
        self.eps = eps
    def forward(self, pred, target):
        return torch.mean(torch.sqrt((pred - target) ** 2 + self.eps ** 2))

def rmse_torch(pred: torch.Tensor, target: torch.Tensor) -> float:
    return float(torch.sqrt(torch.mean((pred - target) ** 2)).item())

def validate_random_patches(model: nn.Module, noisy_list: List[np.ndarray], clean_list: List[np.ndarray], patch: int = 128, samples_per_img: int = 8) -> float:
    model.eval()
    errs = []
    with torch.no_grad(), torch.cuda.amp.autocast(enabled=True):
        for n_img, c_img in zip(noisy_list, clean_list):
            for _ in range(samples_per_img):
                n, c = random_crop_pair(n_img, c_img, patch)
                n_t = torch.from_numpy(n).unsqueeze(0).unsqueeze(0).to(device)
                c_t = torch.from_numpy(c).unsqueeze(0).unsqueeze(0).to(device)
                out = model(n_t)
                errs.append(rmse_torch(out, c_t))
    return float(np.mean(errs)) if errs else math.inf

def train_dncnn(epochs: int = 30, patch: int = 128, train_len: int = 40000, batch: int = 16, lr: float = 1e-3, wd: float = 1e-5, val_frac: float = 0.1, seed: int = 42, patience: int = 6):
    torch.manual_seed(seed); np.random.seed(seed); random.seed(seed)
    print('Preloading train arrays ...')
    ids, noisy_all, clean_all = load_train_arrays()
    tr_idx, val_idx = split_ids(ids, val_frac=val_frac, seed=seed)
    noisy_tr = [noisy_all[i] for i in tr_idx]; clean_tr = [clean_all[i] for i in tr_idx]
    noisy_val = [noisy_all[i] for i in val_idx]; clean_val = [clean_all[i] for i in val_idx]
    print(f'Train images: {len(tr_idx)} | Val images: {len(val_idx)}')
    ds = PatchDataset(noisy_tr, clean_tr, patch_size=patch, length=train_len)
    dl = DataLoader(ds, batch_size=batch, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)
    model = DnCNN(depth=15, n_feats=64).to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=epochs)
    criterion = CharbonnierLoss()
    scaler = torch.cuda.amp.GradScaler(enabled=True)
    best_rmse = float('inf'); best_path = 'dncnn_best.pt'; no_improve = 0
    iters_per_epoch = len(dl)
    for ep in range(1, epochs + 1):
        model.train()
        t0 = time.time(); running = 0.0
        for it, (n, c) in enumerate(dl):
            n = n.to(device, non_blocking=True)
            c = c.to(device, non_blocking=True)
            opt.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=True):
                out = model(n)
                loss = criterion(out, c)
            scaler.scale(loss).backward()
            scaler.step(opt)
            scaler.update()
            running += float(loss.item())
            if (it + 1) % 200 == 0:
                print(f'Ep {ep} Iter {it+1}/{iters_per_epoch} loss {running/(it+1):.5f}', flush=True)
        sched.step()
        tr_loss = running / max(1, iters_per_epoch)
        val_rmse = validate_random_patches(model, noisy_val, clean_val, patch=patch, samples_per_img=8)
        print(f'Epoch {ep}: train_loss {tr_loss:.5f} | val_RMSE {val_rmse:.5f} | lr {sched.get_last_lr()[0]:.6f} | elapsed {time.time()-t0:.1f}s', flush=True)
        if val_rmse < best_rmse - 1e-4:
            best_rmse = val_rmse; no_improve = 0
            torch.save({'state_dict': model.state_dict(), 'epoch': ep, 'val_rmse': best_rmse}, best_path)
            print(f'  Saved new best to {best_path} (RMSE {best_rmse:.5f})')
        else:
            no_improve += 1
        if no_improve >= patience:
            print('Early stopping triggered')
            break
    print('Best val RMSE:', best_rmse)
    return best_rmse

def cosine_window(h: int, w: int) -> torch.Tensor:
    y = 0.5 * (1 - torch.cos(torch.linspace(0, math.pi, steps=h, device=device)))
    x = 0.5 * (1 - torch.cos(torch.linspace(0, math.pi, steps=w, device=device)))
    win = torch.ger(y, x)
    return win

def inference_tiled(model: nn.Module, img: np.ndarray, tile: int = 256, stride: int = 128) -> np.ndarray:
    model.eval()
    H, W = img.shape
    pad_h = (math.ceil((H - tile) / stride) * stride + tile) - H if H > tile else tile - H
    pad_w = (math.ceil((W - tile) / stride) * stride + tile) - W if W > tile else tile - W
    pad_top = pad_h // 2; pad_bottom = pad_h - pad_top
    pad_left = pad_w // 2; pad_right = pad_w - pad_left
    img_pad = np.pad(img, ((pad_top, pad_bottom), (pad_left, pad_right)), mode='reflect')
    Hp, Wp = img_pad.shape
    out = torch.zeros((1, 1, Hp, Wp), device=device)
    weight = torch.zeros((1, 1, Hp, Wp), device=device)
    win = cosine_window(tile, tile).view(1, 1, tile, tile)
    with torch.no_grad(), torch.cuda.amp.autocast(enabled=True):
        for y in range(0, Hp - tile + 1, stride):
            for x in range(0, Wp - tile + 1, stride):
                patch = torch.from_numpy(img_pad[y:y+tile, x:x+tile]).to(device).view(1, 1, tile, tile)
                pred = model(patch)
                out[:, :, y:y+tile, x:x+tile] += pred * win
                weight[:, :, y:y+tile, x:x+tile] += win
    out = out / torch.clamp(weight, min=1e-6)
    out_np = out.squeeze().detach().cpu().numpy()
    out_np = out_np[pad_top:pad_top+H, pad_left:pad_left+W]
    return np.clip(out_np, 0.0, 1.0).astype(np.float32)

def predict_test_dncnn(model_path: str = 'dncnn_best.pt', tile: int = 256, stride: int = 128) -> Dict[str, np.ndarray]:
    test_ids = list_image_ids(TEST_DIR)
    model = DnCNN(depth=15, n_feats=64).to(device)
    ckpt = torch.load(model_path, map_location=device)
    model.load_state_dict(ckpt['state_dict'])
    preds = {}
    t0 = time.time()
    for i, img_id in enumerate(test_ids):
        if i % 5 == 0:
            print(f'Test DL {i}/{len(test_ids)} elapsed {time.time()-t0:.1f}s', flush=True)
        noisy = read_gray_float(os.path.join(TEST_DIR, f'{img_id}.png'))
        pred = inference_tiled(model, noisy, tile=tile, stride=stride)
        preds[img_id] = pred
    return preds

print('DnCNN utilities ready: call train_dncnn(...) to train, then predict_test_dncnn(...) and write with write_submission_streaming_prec.')

DnCNN utilities ready: call train_dncnn(...) to train, then predict_test_dncnn(...) and write with write_submission_streaming_prec.


In [ ]:
# Kick off DnCNN training
print('Starting DnCNN training ...')
best = train_dncnn(epochs=30, patch=128, train_len=60000, batch=16, lr=1e-3, wd=1e-5, val_frac=0.1, seed=42, patience=6)
print('Training complete. Best val RMSE:', best)
print('Best model saved to dncnn_best.pt')

In [ ]:
# Inference with DnCNN best model and write compact submission (explicit per-image logging)
import time, os
from typing import Dict
print('Running DnCNN test inference (tile=512, stride=256) ...', flush=True)
test_ids = list_image_ids(TEST_DIR)
# Load model once
model = DnCNN(depth=15, n_feats=64).to(device)
ckpt = torch.load('dncnn_best.pt', map_location=device)
model.load_state_dict(ckpt['state_dict'])
preds_dl: Dict[str, np.ndarray] = {}
t0 = time.time()
for i, img_id in enumerate(test_ids):
    t_img = time.time()
    noisy = read_gray_float(os.path.join(TEST_DIR, f'{img_id}.png'))
    pred = inference_tiled(model, noisy, tile=512, stride=256)
    preds_dl[img_id] = pred
    print(f'DL pred {i+1}/{len(test_ids)} id={img_id} elapsed_img {time.time()-t_img:.1f}s total {time.time()-t0:.1f}s', flush=True)
print('Inference complete. Writing submission.csv with 3-decimal precision ...', flush=True)
write_submission_streaming_prec(preds_dl, sample_path=SAMPLE_SUB, out_path='submission.csv', precision=3)
print('submission.csv size (MB):', os.path.getsize('submission.csv')/1e6, flush=True)

In [ ]:
# CPU fallback inference for DnCNN with per-image logging and compact submission
import os, time, math
from typing import Dict
import numpy as np
import torch
import torch.nn as nn

def cosine_window_cpu(h: int, w: int) -> torch.Tensor:
    y = 0.5 * (1 - torch.cos(torch.linspace(0, math.pi, steps=h)))
    x = 0.5 * (1 - torch.cos(torch.linspace(0, math.pi, steps=w)))
    return torch.ger(y, x)

def inference_tiled_cpu(model: nn.Module, img: np.ndarray, tile: int = 512, stride: int = 256) -> np.ndarray:
    model.eval()
    device_cpu = torch.device('cpu')
    H, W = img.shape
    pad_h = (max(H, tile) - H)
    pad_w = (max(W, tile) - W)
    pad_top = pad_h // 2; pad_bottom = pad_h - pad_top
    pad_left = pad_w // 2; pad_right = pad_w - pad_left
    img_pad = np.pad(img, ((pad_top, pad_bottom), (pad_left, pad_right)), mode='reflect')
    Hp, Wp = img_pad.shape
    # Ensure grid covers entire image
    ys = list(range(0, max(1, Hp - tile + 1), stride))
    xs = list(range(0, max(1, Wp - tile + 1), stride))
    if ys[-1] != Hp - tile:
        ys.append(max(0, Hp - tile))
    if xs[-1] != Wp - tile:
        xs.append(max(0, Wp - tile))
    out = torch.zeros((1, 1, Hp, Wp), device=device_cpu)
    weight = torch.zeros((1, 1, Hp, Wp), device=device_cpu)
    win = cosine_window_cpu(tile, tile).view(1, 1, tile, tile).to(device_cpu)
    with torch.no_grad():
        for y in ys:
            for x in xs:
                patch = torch.from_numpy(img_pad[y:y+tile, x:x+tile]).view(1, 1, tile, tile).to(device_cpu)
                pred = model(patch)
                out[:, :, y:y+tile, x:x+tile] += pred * win
                weight[:, :, y:y+tile, x:x+tile] += win
    out = out / torch.clamp(weight, min=1e-6)
    out_np = out.squeeze().cpu().numpy()
    out_np = out_np[pad_top:pad_top+H, pad_left:pad_left+W]
    return np.clip(out_np, 0.0, 1.0).astype(np.float32)

print('CPU inference fallback: loading model and running per-image tiling ...', flush=True)
test_ids = list_image_ids(TEST_DIR)
device_cpu = torch.device('cpu')
model_cpu = DnCNN(depth=15, n_feats=64).to(device_cpu)
ckpt = torch.load('dncnn_best.pt', map_location=device_cpu)
model_cpu.load_state_dict(ckpt['state_dict'])
preds_dl: Dict[str, np.ndarray] = {}
t0 = time.time()
for i, img_id in enumerate(test_ids):
    t_img = time.time()
    noisy = read_gray_float(os.path.join(TEST_DIR, f'{img_id}.png'))
    pred = inference_tiled_cpu(model_cpu, noisy, tile=512, stride=256)
    preds_dl[img_id] = pred
    print(f'CPU DL pred {i+1}/{len(test_ids)} id={img_id} elapsed_img {time.time()-t_img:.1f}s total {time.time()-t0:.1f}s', flush=True)
print('CPU inference complete. Writing submission.csv (precision=3) ...', flush=True)
write_submission_streaming_prec(preds_dl, sample_path=SAMPLE_SUB, out_path='submission.csv', precision=3)
print('submission.csv size (MB):', os.path.getsize('submission.csv')/1e6, flush=True)

In [ ]:
# Quick sanity: run DL inference on first 2 test images and print progress
print('Sanity DL inference on first 2 test images ...', flush=True)
test_ids = list_image_ids(TEST_DIR)
print('Test count:', len(test_ids), 'First IDs:', test_ids[:2], flush=True)
model = DnCNN(depth=15, n_feats=64).to(device)
ckpt = torch.load('dncnn_best.pt', map_location=device)
model.load_state_dict(ckpt['state_dict'])
for i, img_id in enumerate(test_ids[:2]):
    t0 = time.time()
    noisy = read_gray_float(os.path.join(TEST_DIR, f'{img_id}.png'))
    pred = inference_tiled(model, noisy, tile=256, stride=128)
    print(f'OK {i+1}/2 id={img_id} shape={pred.shape} elapsed {time.time()-t0:.2f}s', flush=True)
print('Sanity inference finished.', flush=True)

In [8]:
# Simple full-image DnCNN inference (no tiling) + write submission (precision=3)
import os, time
import numpy as np
import torch

print('Running simple full-image DnCNN inference (no tiling) ...', flush=True)
test_ids = list_image_ids(TEST_DIR)
model = DnCNN(depth=15, n_feats=64).to(device)
ckpt = torch.load('dncnn_best.pt', map_location=device)
model.load_state_dict(ckpt['state_dict'])
model.eval()
preds = {}
t0 = time.time()
with torch.no_grad():
    for i, img_id in enumerate(test_ids):
        if i % 5 == 0:
            print(f'  {i}/{len(test_ids)} elapsed {time.time()-t0:.1f}s', flush=True)
        noisy = read_gray_float(os.path.join(TEST_DIR, f'{img_id}.png'))
        inp = torch.from_numpy(noisy).to(device).view(1,1,*noisy.shape)
        out = model(inp).squeeze().detach().cpu().numpy().astype(np.float32)
        preds[img_id] = np.clip(out, 0.0, 1.0)
print('Inference done. Writing submission.csv (precision=3) ...', flush=True)
write_submission_streaming_prec(preds, sample_path=SAMPLE_SUB, out_path='submission.csv', precision=3)
print('submission.csv size (MB):', os.path.getsize('submission.csv')/1e6, flush=True)

Running simple full-image DnCNN inference (no tiling) ...


  0/29 elapsed 0.0s


/tmp/ipykernel_744/2170194961.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load('dncnn_best.pt', map_location=device)


  5/29 elapsed 0.3s


  10/29 elapsed 0.5s


...

Wrote 2000000 rows ...


Wrote 3000000 rows ...


Wrote 4000000 rows ...


Wrote 5000000 rows ...


Submission written to submission.csv
submission.csv size (MB): 104.718754


In [6]:
# Re-define write_submission_streaming_prec to fix NameError without re-running blend pipeline
import csv
from typing import Dict
import numpy as np

def write_submission_streaming_prec(preds: Dict[str, np.ndarray], sample_path: str = SAMPLE_SUB, out_path: str = 'submission.csv', precision: int = 3) -> None:
    fmt = '{:.' + str(precision) + 'f}'
    with open(sample_path, 'r') as fin, open(out_path, 'w', newline='') as fout:
        reader = csv.reader(fin)
        writer = csv.writer(fout)
        header = next(reader)
        writer.writerow(header)
        line_ct = 0
        for row in reader:
            id_str = row[0]
            parts = id_str.split('_')
            if len(parts) != 3:
                continue
            img_id, r_str, c_str = parts
            r = int(r_str) - 1
            c = int(c_str) - 1
            arr = preds.get(img_id)
            if arr is None:
                raise KeyError(f'Missing prediction for image id {img_id}')
            val = float(np.clip(arr[r, c], 0.0, 1.0))
            writer.writerow([id_str, fmt.format(val)])
            line_ct += 1
            if line_ct % 1000000 == 0:
                print(f'Wrote {line_ct} rows ...', flush=True)
    print(f'Submission written to {out_path}')